In [2]:
import sympy
from scipy import optimize
import numpy as np
import plotly.graph_objects as go

In [16]:
class Optimization:
    """
    q1:param , q2:param - функции спроса на товар x1 и x2 соответсвенно\n
    C:param - функция издержек\n
    params_dict:param - словарь параметров, которые нужно подставить в функции
    """

    def __init__(self, q1:str, q2:str, C:str, params_dict:dict=None):
        # Запоминаем исходные входные значения
        if params_dict is None:
            params_dict = {}
        self._q1_source = q1
        self._q2_source = q2
        self._C_source = C
        self._params_source = params_dict
        self._P_source = "p_1 * q_1 + p_2 * q_2 - C"



        # Формируем падстановку выражений
        self.p1 = sympy.Symbol('p_1')
        self.p2 = sympy.Symbol('p_2')
        self.q1 = sympy.parse_expr(q1)
        self.q2 = sympy.parse_expr(q2)
        self.C = sympy.parse_expr(C).subs({'q_1': f'({self.q1})', 'q_2': f'({self.q2})'})
        self.P = self.p1 * self.q1 + self.p2 * self.q2 - self.C

        # Подстановка параметров
        self.substitution()

        self.optimize_p1, \
        self.optimize_p2, \
        self.optimize_P = self.find_max()

        sub = lambda x : sympy.Symbol.subs(x, {'p_1': self.optimize_p1, 'p_2': self.optimize_p2})
        self.optimize_q1 = float(round(sub(self.q1)))
        self.optimize_q2 = float(round(sub(self.q2)))
        self.optimize_C = sub(self.C)

        self.surface_plot, self.contour_plot = self._create_plots()


    def _create_data_P(self):
        x = np.linspace(1, 10, 10)
        y = np.linspace(1, 10, 10)

        P = sympy.parse_expr(self._P_source)\
            .subs({'C': self._C_source})\
            .subs({'p_1': self.optimize_p1, 'p_2': self.optimize_p2})\
            .subs(self._params_source)

        P = sympy.lambdify(['q_1', 'q_2'], P) # Создание функции


        X, Y = np.meshgrid(x, y) # Определение координатных векторов
        Z = np.vectorize(P)(X, Y) # Векторизуем функцию и подадим массивы в качестве параметров

        return x,y,Z


    def _create_plots(self):
        scene = go.Scene(
        xaxis = {'title': 'q1 - спрос на первый товар'},
        yaxis = {'title': 'q2 - спрос на второй товар'},
        zaxis = {'title': 'прибыль'})

        layout = go.Layout(scene=scene)

        x, y, Z = self._create_data_P()

        return self._create_surface_plot(x,y,Z, layout), \
               self._create_contour_plot(x,y,Z, layout)

    def substitution(self):
        uni_subs = lambda x: sympy.Symbol.subs(x, self._params_source)

        self.q1 = uni_subs(self.q1)
        self.q2 = uni_subs(self.q2)
        self.C = uni_subs(self.C)
        self.P = uni_subs(self.P)

    def _get_lambdify_P(self):
        return sympy.lambdify(['p_1', 'p_2'], self.P)


    def _create_surface_plot (self, x, y, Z, layout):
        surf = go.Surface(x=x, y=y, z=Z)
        scatter_3d = go.Scatter3d(x=[self.optimize_q1],
                          y=[self.optimize_q2],
                          z=[self.optimize_P],
                          mode="markers+text",
                          text=[f'M* ({self.optimize_q1},{self.optimize_q2})']
                          )

        fig = go.Figure(data=[surf, scatter_3d], layout=layout)
        return fig


    def _create_contour_plot(self, x, y, Z, layout):
        contour = go.Contour(x=x, y=y, z=Z)
        fig = go.Figure(data=[contour], layout=layout)

        fig.add_trace(go.Scatter(
            x=[self.optimize_q1],
            y=[self.optimize_q2],
            mode="markers+text",
            textposition="bottom center",
            text= [f'M* ({self.optimize_q1},{self.optimize_q2})']))

        return fig

    def __repr__(self):
        return f"""Оптимизированные значения:
        p1 = {self.optimize_p1}
        p2 = {self.optimize_p2}
        q1 = {self.optimize_q1}
        q2 = {self.optimize_q2}
        C = {self.optimize_C}
        P = {self.optimize_P}
"""

    """
    Возвращает точку максимума функии P, а также ее максимум -> (p1, p2, max)
    """
    def find_max(self):

        res = optimize.minimize(lambda x: -self.lambdified_P(x[0], x[1]), (0, 0))
        return res.x[0], res.x[1], -res.fun


    lambdified_P = property(fget=_get_lambdify_P)




In [17]:
rep_d =  {
        'a': 80,
        'b': -2,
        'c': 58,
        'd': -1,
        'k': 2,
        'l': 5,
        'm': 3,
        'n': 7
    }

opt = Optimization(
    "a + b * p_1",
    "c + d * p_2",
    "k * q_1 ** 2 + l * q_1 * q_2 + m * q_2 ** 2 + n", rep_d)


opt.P

p_1*(80 - 2*p_1) + p_2*(58 - p_2) - 3*(58 - p_2)**2 - 5*(58 - p_2)*(80 - 2*p_1) - 2*(80 - 2*p_1)**2 - 7

In [18]:
opt

Оптимизированные значения:
        p1 = 38.99999962733332
        p2 = 52.000000954967604
        q1 = 2.0
        q2 = 6.0
        C = 182.999984394158
        P = 206.99999999999852

In [19]:
opt.surface_plot

In [236]:
opt.contour_plot